In [11]:
# when installing the libraries after cloning the repository, use this command: pip install -r requirements.txt
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

import setuptools.dist

from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, LSTM
from keras.utils import pad_sequences

In [17]:
train_data = pd.read_csv("train.csv")

In [ ]:
import re
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

def add_hashtags_column(df):
    new_df = df.copy()

    # Extract words that start with '#' and remove the '#' symbol and remove also special characters and numbers
    new_df['hashtags'] = new_df['text'].apply(
        lambda x: [re.sub(r'[^a-zA-Z]', '', re.sub(r'^#+', '', word)) for word in x.split() if word.startswith('#')]
    )

    return new_df

def get_sentiment(text):
    # Initialize VADER sentiment intensity analyzer
    sia = SentimentIntensityAnalyzer()

    sentiment_score = sia.polarity_scores(text)

    if sentiment_score['compound'] >= 0.05:
        return 'Positive'
    elif sentiment_score['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'


In [20]:
hashtagged_data = add_hashtags_column(train_data)
hashtagged_data['sentiment'] = hashtagged_data['text'].apply(get_sentiment)